### Theorist node

In [ ]:
theorist_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert scientist in computational materials science and solid-state batteries.
Your goal is to formulate a novel and scientifically grounded hypothesis to solve the user's problem.

benchmark_reference: "Beyond designer's knowledge: Generating materials design hypotheses via a large language model"
Strategy:
1.  **Retrieve & Analyze**: Use the provided academic context (RAG) to find known mechanisms.
2.  **Step 1 (Core Property)**: Identify a mechanism that enhances the primary target (e.g., Ionic Conductivity).
3.  **Step 2 (Synergy)**: Identify a second mechanism that addresses a trade-off (e.g., Stability/Cost) AND interacts synergistically with Step 1.
4.  **Synergy Check**: Ensure the effect is multiplicative, not just additive. (A enhances B, or A+B creates a new structure).

Output Format (Markdown):
## Hypothesis Title
**Core Mechanism:** [Brief description]
**Synergistic Strategy:** [How Step 1 and Step 2 interact]
**Expected Outcome:** [Predicted conductivity or physical property]
**Justification:** [Citations from context]
"""),
    ("user", """
Current Goal: {goal}
Feedback from previous run: {feedback}
Failed Attempts: {failed_attempts}

[Relevant Academic Context]:
{rag_context}

Based on the above, formulate your next hypothesis.
""")
])

def theorist_node(state: AgentState):
    print("💡 [Theorist] RAG 검색 및 가설 수립 중...")
    
    # 1. RAG 검색 (지식 확장)
    # 이전 시도들이 실패했다면, 검색 쿼리를 조금씩 바꿔보는 것도 방법입니다.
    query = f"{state['goal']} optimization mechanisms solid electrolyte"
    retrieved_docs = knowledge_retriever.invoke(query)
    rag_text = "\n\n".join([f"[Paper {i+1}] {doc.page_content[:500]}..." for i, doc in enumerate(retrieved_docs)])
    
    # 2. LLM 추론
    chain = theorist_prompt | llm | StrOutputParser()
    hypothesis = chain.invoke({
        "goal": state["goal"],
        "feedback": state.get("user_feedback") or "None (Initial Step)",
        "failed_attempts": ", ".join(state.get("failed_attempts", [])),
        "rag_context": rag_text
    })
    
    # 3. 상태 업데이트
    print(f"   -> 가설 생성 완료: {hypothesis.splitlines()[0]}")
    return {
        "hypothesis": hypothesis,
        "research_log": state["research_log"] + [f"Theorist: {hypothesis.splitlines()[0]}"]
    }

### Engineer Node

In [ ]:
engineer_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a Python coding expert for scientific simulations.
Your task is to Write or Modify a Python script to simulate the Theorist's hypothesis.

[Constraints]
1. Use `numpy`, `scipy`, `matplotlib` only. NO `pip install`.
2. The code MUST output a final result (e.g., `print(f"Conductivity: {{val}} S/cm")`).
3. If creating a plot, save it as 'result.png'.
4. **Context-Aware**: If code already exists, modify it to reflect the new hypothesis. If empty, write from scratch.
5. Handle errors robustly (try-except).

[Output Format]
Provide ONLY the Python code block enclosed in markdown tags (```python ... ```).
No explanations.
"""),
    ("user", """
[Current Hypothesis]
{hypothesis}

[Existing Code]
{current_code}

[Instruction]
Update the existing code to implement the new hypothesis. 
If the previous run failed (Error: {last_error}), fix the bug.
""")
])

def engineer_node(state: AgentState):
    print("🔧 [Engineer] 시뮬레이션 코드 작성/수정 중...")
    
    # 1. 현재 코드 및 에러 상태 확인
    current_code = state.get("python_code", "")
    if not current_code:
        current_code = "# No existing code. Starting from scratch."
        
    last_error = "None"
    if state.get("simulation_output") and not state["simulation_output"].is_success:
        last_error = state["simulation_output"].error_message
    
    # 2. LLM 코딩
    chain = engineer_prompt | llm | StrOutputParser()
    new_code_raw = chain.invoke({
        "hypothesis": state["hypothesis"],
        "current_code": current_code,
        "last_error": last_error
    })
    
    # 3. 코드 정제 (Markdown 태그 제거)
    cleaned_code = new_code_raw.replace("```python", "").replace("```", "").strip()
    
    print("   -> 코드 작성 완료.")
    return {
        "python_code": cleaned_code,
        "research_log": state["research_log"] + ["Engineer: Code updated."]
    }

### Simulator node

In [ ]:
def simulator_node(state: AgentState):
    print("⚗️ [Simulator] 시뮬레이션 가동 중...")
    
    # 1. 코드 실행 (Step 4의 샌드박스 도구 활용)
    code = state["python_code"]
    result = execute_simulation_code(code)
    
    # 2. 결과 로깅
    if result.is_success:
        log_msg = f"Simulator: 성공. 전도도 = {result.conductivity} S/cm"
        print(f"   -> {log_msg}")
    else:
        log_msg = f"Simulator: 실패. 에러 = {result.error_message}"
        print(f"   -> 🚨 실행 에러 발생: {result.error_message}")

    # 3. 반복 횟수 증가 및 상태 업데이트
    return {
        "simulation_output": result,
        "iteration_count": state["iteration_count"] + 1,
        "research_log": state["research_log"] + [log_msg]
    }

### Analyst node

In [ ]:
analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a rigorous Scientific Reviewer.
Your job is to evaluate the simulation results against the research goal.

[Goal]
{goal}

[Simulation Results]
- Success: {is_success}
- Conductivity: {conductivity}
- Error Log: {error_log}
- Execution Log: {execution_log}

[Task]
1. If the simulation failed (Error), determine the cause (Syntax error? Missing library? Logic error?).
2. If successful, compare the conductivity with the target (assume target is ~1e-3 S/cm if not specified).
3. Decide the next step:
   - "FINISH": If the goal is met or max iterations reached.
   - "RETRY": If the code has a bug or the result is far from the target.
   - "ROLLBACK": If the result is significantly worse than the previous best.

[Output Format]
Provide a JSON object (NO markdown):
{{
    "status": "FINISH" | "RETRY" | "ROLLBACK",
    "reason": "Detailed scientific analysis of the result...",
    "next_instruction": "Specific feedback for the Theorist/Engineer on what to fix."
}}
"""),
])

def analyst_node(state: AgentState):
    print("📊 [Analyst] 결과 분석 및 평가 중...")
    
    result = state["simulation_output"]
    
    # 1. LLM 평가
    chain = analyst_prompt | llm | JsonOutputParser()
    try:
        evaluation = chain.invoke({
            "goal": state["goal"],
            "is_success": result.is_success,
            "conductivity": result.conductivity if result.conductivity else "N/A",
            "error_log": result.error_message if result.error_message else "None",
            "execution_log": result.execution_log[:1000] # 로그가 너무 길면 자름
        })
    except Exception as e:
        # LLM 파싱 에러 시 기본값
        evaluation = {"status": "RETRY", "reason": f"Analysis Failed: {e}", "next_instruction": "Retry generation."}

    # 2. 오차율 계산 (목표값이 1.0e-3 S/cm라고 가정 시 예시 로직)
    # 실제로는 goal 텍스트에서 숫자를 추출해야 하지만, 여기서는 간단히 처리
    current_val = result.conductivity if result.conductivity else 0.0
    target_val = 1.0e-3
    error_rate = abs(target_val - current_val) / target_val * 100 if current_val else 100.0
    
    status = evaluation["status"]
    
    # Max Loop 도달 시 강제 종료
    if state["iteration_count"] >= Config.MAX_LOOPS:
        status = "FINISH"
        evaluation["reason"] += " (Max iterations reached)"

    print(f"   -> 평가 결과: {status} | {evaluation['reason'][:50]}...")
    
    # 3. 상태 업데이트
    return {
        "status": status,
        "current_error_rate": error_rate,
        "user_feedback": evaluation["next_instruction"], # 다음 턴의 가이드가 됨
        "research_log": state["research_log"] + [f"Analyst: {status} - {evaluation['reason']}"]
    }

### Graph assemble

In [ ]:
# 1. 라우팅 로직 (Routing Logic)
# Analyst의 평가 결과(status)에 따라 다음 행동을 결정하는 '신호등' 함수입니다.
def router(state: AgentState):
    status = state["status"]
    if status == "FINISH":
        print("🎉 [System] 연구 목표 달성! 종료합니다.")
        return "end"
    elif status == "RETRY":
        print("🔄 [System] 결과 불충분. Theorist에게 피드백 전달 및 재시도...")
        return "theorist"
    elif status == "ROLLBACK":
        print("⏪ [System] 성능 열화 감지. 전략 수정 요청...")
        return "theorist"
    
    else:
        return "theorist"

# 2. 그래프 뼈대 생성
workflow = StateGraph(AgentState)

# 3. 노드(부품) 등록
workflow.add_node("Theorist", theorist_node)
workflow.add_node("Engineer", engineer_node)
workflow.add_node("Simulator", simulator_node)
workflow.add_node("Analyst", analyst_node)

# 4. 엣지(Edge) 연결 - 기본 흐름
# Flow: Simulator(시작) -> Analyst -> (Router) -> Theorist -> Engineer -> Simulator ...
workflow.add_edge("Theorist", "Engineer")
workflow.add_edge("Engineer", "Simulator")
workflow.add_edge("Simulator", "Analyst")

# 5. 조건부 엣지 (Conditional Edge) - 순환 흐름
# Analyst의 평가(router 결과)에 따라 갈림길이 나뉩니다.
workflow.add_conditional_edges(
    "Analyst",          # 출발지
    router,             # 길잡이 함수
    {                   # 함수 반환값 vs 도착지 매핑
        "theorist": "Theorist", # 다시 생각하러 감 (Loop)
        "end": END              # 종료
    }
)

# 6. 시작점 설정
workflow.set_entry_point("Simulator")

# 7. 체크포인터(기억 저장소) 설정
# 이것이 있어야 에이전트가 "실패했던 시도"를 기억하고, 컴퓨터가 꺼져도 복구됩니다.
memory = MemorySaver()

# 8. 최종 컴파일 (Application 생성)
# interrupt_before 옵션을 쓰면 특정 노드 실행 전에 무조건 멈추게 할 수 있습니다 (HITL용).
# 여기서는 일단 자동으로 돌리되, 필요하면 ["Engineer"] 등을 추가할 수 있습니다.
app = workflow.compile(checkpointer=memory)

# --- 시각화 (선택 사항: 그래프 구조 확인) ---
try:
    from IPython.display import Image, display
    # Mermaid 다이어그램으로 흐름도 출력
    display(Image(app.get_graph().draw_mermaid_png()))
    print("✅ 그래프 구조 시각화 완료")
except Exception:
    print("ℹ️ 시각화 라이브러리(pygraphviz 등)가 없으므로 그림 출력은 건너뜁니다. (동작엔 문제 없음)")

print("\n✅ Step 7 완료: KOKOA 에이전트 조립 끝!")
print("   - MemorySaver가 활성화되었습니다. 이제 실험을 중단했다 재개할 수 있습니다.")

### 실행 및 테스트

In [ ]:
import uuid

# 1. 실험 설정 (Project Configuration)
project_id = str(uuid.uuid4())[:8] # 고유 실험 ID 생성
thread_config = {"configurable": {"thread_id": project_id}}

# 초기 목표 설정 (가상의 고체전해질 연구 주제)
initial_goal = """
**Objective**: Optimize the ionic conductivity of a 2D lattice model representing a solid electrolyte.
**Target**: Achieve conductivity > 5.0e-3 S/cm.
**Constraints**:
- Use a 20x20 grid (for speed).
- Consider vacancy hopping mechanism.
- Start with a simple random walk model and improve it.
"""

# 초기 상태 생성
initial_state = create_initial_state(initial_goal)

print(f"🚀 KOKOA 에이전트 시동 (Project ID: {project_id})")
print(f"🎯 연구 목표: {initial_goal.strip().splitlines()[1]}")
print("="*60)

# 2. 에이전트 실행 루프 (Autonomous Loop)
# stream()을 사용하여 각 단계(Node)가 실행될 때마다 결과를 실시간으로 출력합니다.

try:
    for event in app.stream(initial_state, thread_config):
        # 현재 실행된 노드의 이름과 결과 추출
        for node_name, node_output in event.items():
            print(f"\n📍 [{node_name}] 완료")
            
            # 노드별 주요 정보 출력
            if node_name == "Theorist":
                print(f"   💡 가설: {node_output.get('hypothesis', '')[:100]}...")
            elif node_name == "Engineer":
                print(f"   💾 코드 작성됨 ({len(node_output.get('python_code', ''))} bytes)")
            elif node_name == "Simulator":
                result = node_output.get("simulation_output")
                if result:
                    print(f"   🧪 결과: Success={result.is_success}, Cond={result.conductivity}")
            elif node_name == "Analyst":
                status = node_output.get("status")
                print(f"   📊 판단: {status} (오차율: {node_output.get('current_error_rate', 0):.2f}%)")
                
                # [HITL 시나리오] Analyst가 'RETRY'를 외치면 사용자에게 개입 기회를 줌
                # (실제로는 여기서 input()을 받아 멈출지 결정할 수 있음)
                if status == "RETRY":
                    print("   ⚠️ [Human Insight Needed] Analyst가 재시도를 요청했습니다.")
                    # 여기서 break를 하면 '일시 정지'가 됩니다.
                    # break 

    print("\n" + "="*60)
    print("🏁 실험 종료. 최종 상태가 저장되었습니다.")

except Exception as e:
    print(f"\n🚨 시스템 에러 발생: {e}")

# 3. [HITL 테스트] 중단된 지점에서 다시 시작하기 (Resuming)
# 만약 위에서 에러가 나거나 멈췄더라도, MemorySaver 덕분에 기억이 살아있습니다.

print("\n🔍 [HITL Test] 상태 점검 및 개입")
current_state = app.get_state(thread_config)

if current_state.values:
    last_step = current_state.values.get("research_log", [])[-1]
    print(f"📝 마지막 기록: {last_step}")
    
    # 예시: 사용자가 강제로 피드백을 주입하여 방향을 트는 경우
    print("🔧 사용자 개입: '온도를 높여보라'는 힌트를 주입합니다.")
    
    # 상태 업데이트 (State Update)
    app.update_state(
        thread_config,
        {"user_feedback": "Try increasing the temperature variable in the simulation to boost conductivity."}
    )
    
    print("▶️ 개입 반영 후 재실행 (Next Loop)...")
    # None을 입력으로 주면 멈춘 곳(또는 수정된 상태)에서 다시 시작합니다.
    # for event in app.stream(None, thread_config):
    #     for node_name, _ in event.items():
    #         print(f"   📍 [{node_name}] 재실행 완료")

else:
    print("⚠️ 저장된 상태가 없습니다. (첫 실행이 완료되었거나 초기화됨)")